<a href="https://colab.research.google.com/github/with2say/layout2inspection/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -rf layout2inspection
!git clone https://github.com/with2say/layout2inspection.git

Cloning into 'layout2inspection'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 18 (delta 5), reused 10 (delta 3), pack-reused 0
Unpacking objects: 100% (18/18), 7.31 KiB | 1.83 MiB/s, done.


In [ ]:
!pip install lightning --quiet

In [ ]:
import sys
sys.path.append("layout2inspection")

from msp.utils import *
from msp.dataset import *
from msp.model import *
from msp.polytoimage import *

In [ ]:


def main(dataset_kwargs={}, n_epoch=200, model_kwargs={}):
    # input params
    n_samples = dataset_kwargs['n_samples']
    n_channels = dataset_kwargs['n_channels']
    range_shape = dataset_kwargs['range_shape']
    range_polygon = dataset_kwargs['range_polygon']
    n_positions = dataset_kwargs['n_positions']
    n_outputs = dataset_kwargs['n_outputs']

    # checkpoint_path = '/workspaces/layout2inspection/lightning_logs/version_15/checkpoints/epoch=21-step=704.ckpt'

    # addtional params
    n_shapes = np.max(range_shape)
    n_polygons = np.max(range_polygon)

    # hyper params
    # data, targets = generate_data_with_negative_padding(n_samples, range_shape, range_polygon)
    data, targets = generate_dataset(n_samples, n_channels, range_shape, range_polygon)
    print('null mse:', np.var(targets))
    print(data.shape, targets.shape)

    data_module = PolygonAreaDataModule(data, targets, 
                                        batch_size=128, val_split=0.1, test_split=0.1, aug_prob=0.5, num_workers=2)

    # MultiShapeEmbedding 객체 생성
    layer = MultiShapeEmbedding(
        n_positions, n_polygons, n_shapes, n_channels, n_outputs,
        **model_kwargs,
    )

    model = PolygonRegressor(layer)
    # model.load_from_checkpoint(checkpoint_path)

    trainer = get_trainer(n_epoch)
    trainer.fit(model, data_module)
    trainer.validate(model, datamodule=data_module)
    trainer.test(model, datamodule=data_module)

    y_true, y_pred = get_predictions(trainer, model, data_module)
    print(np.shape(y_true), np.shape(y_pred))
    evaluate_regression(y_true, y_pred)
    plot_true_vs_predicted(y_true, y_pred)


if __name__ == '__main__':
    dataset_kwargs = {
        'n_samples': 10000,
        'n_channels': 2,
        'range_shape': [1, 3],
        'range_polygon': [3, 5],
        'n_positions': 2,
        'n_outputs': 1,
    }

    max_epoch = 200

    model_kwargs = {
        'polygon_heads': 4, 'polygon_dimension_per_head': 4, 'polygon_layers': 3, 
        'spatial_output_height': 16, 'spatial_output_width': 16, 
        'shape_output_channels': [32, 64, 128],
        'fc_dimensions': 128, 'fc_layers': 2, 'fc_use_batchnorm': False,
    } # 0.9

    # dataset_kwargs = {
    #     'n_samples': 10000,
    #     'n_channels': 3,
    #     'range_shape': [1, 6],
    #     'range_polygon': [3, 10],
    #     'n_positions': 2,
    #     'n_outputs': 1,
    # }

    # model_kwargs = {
    #     'polygon_heads': 4, 'polygon_dimension_per_head': 4, 'polygon_layers': 3, 
    #     'spatial_output_height': 16, 'spatial_output_width': 16, 
    #     'shape_output_channels': [32, 64, 128],
    #     'fc_dimensions': 128, 'fc_layers': 2, 'fc_use_batchnorm': False,
    # } # 0.9

    main(
        dataset_kwargs=dataset_kwargs, 
        n_epoch=max_epoch,
        model_kwargs=model_kwargs,
    )    

In [ ]:

def main(dataset_kwargs={}, n_epoch=200, model_kwargs={}):
    # input params
    n_samples = dataset_kwargs['n_samples']
    n_channels = dataset_kwargs['n_channels']
    range_shape = dataset_kwargs['range_shape']
    range_polygon = dataset_kwargs['range_polygon']
    n_positions = dataset_kwargs['n_positions']
    n_outputs = dataset_kwargs['n_outputs']

    # checkpoint_path = '/workspaces/layout2inspection/lightning_logs/version_15/checkpoints/epoch=21-step=704.ckpt'

    # addtional params
    n_shapes = np.max(range_shape)
    n_polygons = np.max(range_polygon)

    # hyper params
    # data, targets = generate_data_with_negative_padding(n_samples, range_shape, range_polygon)
    data, targets = generate_dataset(n_samples, n_channels, range_shape, range_polygon)
    print('null mse:', np.var(targets))
    print(data.shape, targets.shape)

    data_module = PolygonAreaDataModule(data, targets, batch_size=256, val_split=0.1, test_split=0.1, num_workers=2, aug_prob=0.3)

    # MultiShapeEmbedding 객체 생성
    # layer = MultiShapeEmbedding(
    #     n_positions, n_polygons, n_shapes, n_channels, n_outputs,
    #     **model_kwargs,
    # )
    layer = SimpleEmbedding(
        n_positions, n_polygons, n_shapes, n_channels, n_outputs,
        **model_kwargs,
    )

    model = PolygonRegressor(layer)
    # model.load_from_checkpoint(checkpoint_path)

    # trainer = get_trainer(n_epoch)
    trainer = pl.Trainer(max_epochs=n_epoch, log_every_n_steps=1)
    trainer.fit(model, data_module)
    trainer.validate(model, datamodule=data_module)
    trainer.test(model, datamodule=data_module)

    y_true, y_pred = get_predictions(trainer, model, data_module)
    print(np.shape(y_true), np.shape(y_pred))
    evaluate_regression(y_true, y_pred)
    plot_true_vs_predicted(y_true, y_pred)


if __name__ == '__main__':
    dataset_kwargs = {
        'n_samples': 20,
        'n_channels': 2,
        'range_shape': [1, 3],
        'range_polygon': [3, 5],
        'n_positions': 2,
        'n_outputs': 1,
    }
    
    model_kwargs = {
        'fc_dimensions': 64, 'fc_layers': 5, 'fc_use_batchnorm': True,
    }
    
    main(
        dataset_kwargs=dataset_kwargs,
        n_epoch=3,
        model_kwargs=model_kwargs,
    )    

In [ ]:

    #d_model=128, n_head=32, n_shapes=[1,1], n_polygons=[3,12], n_epoch50, R2 0.34
    #d_model=128, n_head=32, n_shapes=[1,1], n_polygons=[3,12], n_epoch100, R2 0.63
    #d_model=128, n_head=32, n_shapes=[1,1], n_polygons=[3,12], n_epoch200, R2 0.67
    #d_model=128, n_head=64, n_shapes=[1,1], n_polygons=[3,12], n_epoch200, R2 0.738
    #d_model=256, n_head=64, n_shapes=[1,1], n_polygons=[3,12], n_epoch200, R2 0.70

    #d_model=16, n_head=8, n_layers=3, with_normlayer, n_shapes=[1,1], n_polygons=[3,12], n_epoch200, R2 0.73
    